# SEP Forecasting 
***

Current physics based models of SEPs are unable to execute sufficiently fast in order to
provide actionable information towards forecasting such disturbances, which can impact
Earth within tens of minutes of the onset of an eruptive event. This is compounded by
the intrinsic latency of certain key observations, which are used to define the initial
conditions of these models. Instead, there is a reliance on statistical models to provide
forecast probabilities of Earth-bound SEPs using real-time data. Since the largest, most
impactful events occur infrequently, some regions of the feature space are sparse and
simple discrete binning procedures have limitations. The goal of this project is to
improve upon the empirical SEP proton prediction forecast model (PROTONS) currently
in operational use at SWPC, through the application of modern machine learning
techniques.

In [17]:
# Test comment by Hazel
# ... and another 
# a third from Eric
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

from datetime import datetime,timedelta

from pandas.plotting import scatter_matrix

from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, recall_score, matthews_corrcoef
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, roc_curve, brier_score_loss, precision_score
from sklearn.metrics import brier_score_loss as bsl
from sklearn.metrics import mean_squared_error as mse
from sklearn import preprocessing
from sklearn.utils.class_weight import compute_class_weight
#import scikitplot as skplt

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier as trees



from scipy.sparse import csr_matrix, find


### Read in and format the original Balch 2008 event list 

Choice of files: 

SPEall.v7p.xls - original SEP (only) event list from Balch 2008  
ctrlevents.v8p.xls - original event list (SEP + control) from Balch 2008  
ControlEvents_student.xls - Balch 2008 (SEP + control) event list with added CME speed and GOES T and EM from Dougs student

In [ ]:
#read in the original Balch SEP event list with added CME speed and GOES T and EM
#orig_data = pd.read_excel("ctrlevents.v8p.xls")

#adding y label column indicating positive and negative SEP events - SEP events have Association = ProtonFlare
#orig_data['sep'] = orig_data.Association.str.contains('^Proton').astype(int)

#convert the type II and type IV association to binary 
#orig_data.TypeII = (orig_data.TypeII.str.lower() == "yes").astype(int)
#orig_data.TypeIV = (orig_data.TypeIV.str.lower() == "yes").astype(int)

# Remove rows where optlocation = nan
#orig_data = orig_data[orig_data.optlocation.astype('str') != 'nan']

#shuffle the events so they are not organized 
#orig_data = shuffle(orig_data)

#save the shuffled dataframe -- commented out to prevent resaving
#orig_data.to_csv("AllEvtsShuffled_1986_2004.csv")


In [ ]:
#read in the students version Balch SEP event list with added CME speed and GOES T and EM
#orig_data = pd.read_excel("ControlEvents_student.xls")

#adding y label column indicating positive and negative SEP events - SEP events have Association = ProtonFlare
#orig_data['sep'] = orig_data.Association.str.contains('^Proton').astype(int)

#convert the type II and type IV association to binary 
#orig_data.TypeII = (orig_data.TypeII.str.lower() == "yes").astype(int)
#orig_data.TypeIV = (orig_data.TypeIV.str.lower() == "yes").astype(int)

# Remove rows where optlocation = nan
#orig_data = orig_data[orig_data.optlocation.astype('str') != 'nan']

#shuffle the events so they are not organized 
#orig_data = shuffle(orig_data)

#save the shuffled dataframe -- commented out to prevent resaving
#orig_data.to_csv("AllEvtsShuffled_1986_2004_cme.csv")



### Read in the new SEP events from 2004 onwards

Created by Pedro Brae

In [ ]:
#read in the SEP event list from 2004 - 2017
#new_data = pd.read_csv('sep_events_2004_2017.csv')


### Original Balch Features

From the original Balch 1999 and 2008 paper analysis only 4 features were used. GOES flare peak soft X-ray peak (1-8 A channel), flare soft X-ray integrated flux, occurrence of Type II and Type IV radio bursts.

In [26]:
class BalchPaperFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        """Class to create original 4 features from Balch 2008"""
        
        return None
    
    def fit(self, examples):
        # return self and nothing else 
        return self
    
    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""
        
        return ['FlrPeakFlux','FlrIntFlux2','TypeII','TypeIV']
    
    def transform(self, examples):
                
        #Choose the orginal 4 Balch 2008 features
        X = examples[['FlrPeakFlux','FlrIntFlux2','TypeII','TypeIV']]
        
        return(X)

### Raw features

Other features read directly from the original event list i.e. the flare integrated soft X-ray flux at the GOES short wavelength channel (0.5 - 1 A), the GOES temperature and GOES emission measure

In [27]:
class RawFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        """Other features from the Event List that don't need to be manipulated before being included in model"""
        
        return None
    
    def fit(self, examples):
        # return self and nothing else 
        return self
    
    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""
        
        return ['FlrIntFlux','tchianti','emchianti']
    
    def transform(self, examples):
                
        #Choose the orginal 4 Balch 2008 features (5 if you include both type II and type Iv as separate features)
        X = examples[['FlrIntFlux','tchianti','emchianti']]

        return(X)

### Flare time to peak

The time between the flare onset and the flare peak time

In [28]:
class FlareTime2Peak(BaseEstimator, TransformerMixin):
    
    """Class to create feature with the time between flare onset to flare max time"""
    
    def __init__(self):
        
        return None
    
    def fit(self, examples):
        # return self and nothing else 
        return self
    
    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""
        
        return ["FlTim2Pk"]
    
    def transform(self, examples):
        
        # Initiaize matrix 
        X = np.zeros((len(examples),1))           

        #time between flare max and flare onset
        X[:,0] = np.asarray([(mx - on).seconds for mx, on in zip(examples.Flrmaxtime, examples.FlrOnset)])
        
        return(X)

### Flare location

The flare location formatted from N16W12 format to North-South and East-West of disk center values

In [29]:
class LocationFeatures(BaseEstimator, TransformerMixin):
   
    """Class to create feature with the time between flare onset to flare max time"""
   
    def __init__(self):
       
        return None
   
    def fit(self, examples):
        # return self and nothing else 
        return self
   
    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""
       
        return ['NS', 'EW']
   
    def transform(self, examples):
       
        # Initiaize matrix 
        X = np.zeros((len(examples),2))  
       
        #time between flare max and flare onset
        #X[:,0] = np.asarray([str(x)[1:3] for x in examples.optlocation])   #north - south
        #X[:,1] = np.asarray([str(x)[4::] for x in examples.optlocation])   #north - south
       
        for i,loc in enumerate(examples.optlocation):
            lat = str(loc)[1:3]
            if str(loc)[0] == 's' or str(loc)[0] == 'S':
                X[i,0] = -int(lat)
            else:
                X[i,0] = (lat)

            #west-east
            long = str(loc)[4::]
            if str(loc)[3] == 'w' or str(loc)[3] == 'W':
                X[i,1] = -int(long)
            else:
                X[i,1] = int(long)

        return(X)

### SEP Class 

With functions to build the training feature list from the above feature extractors

In [30]:
class SEPClass(BaseEstimator, ClassifierMixin):
    def __init__(self, estimator, folds = 6, threshold=0.5):        # <--- other keywords to be used by Feature Union go here
        
        """Class to fit and train Logistic Regression algorithm for SEP forecasting
        
        Input keywords:
        
        folds:        Number of cross validation folds to use
        threshold:    Decision Boundary threshold
        
        """
        
        self.estimator = estimator     #estimator to use for classification e.g. LogReg or SVM
        self.folds = folds             #cross validation folds for estimator
        self.threshold = threshold     #decision boundary threshold
    
        #Set up the Feature union to combine Feature creating classes
        self.allmyfeatures = FeatureUnion([
            ("BalchFeat", BalchPaperFeatures()),
            ("RawFeat", RawFeatures()),
            ("LocFeatures", LocationFeatures())#,
            #("FlareTime2Peak", FlareTime2Peak())   
        ])
    
    def build_train_features(self, examples):
        """
        Method to take in training text features and do further feature engineering 
        Most of the work in this homework will go here, or in similar functions  
        :param examples: currently just a list of forum posts  
        """  
        
        ##convert columns of time from string to datetime -- MOVE TO SEPARATE CLEANING FUNCTION
        #examples.FlrOnset = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.FlrOnset)
        #examples.Flrmaxtime = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.Flrmaxtime)
        #examples.Flrendtime = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.Flrendtime)
        #examples.FlrHpTime = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.FlrHpTime)
                
        return self.allmyfeatures.fit_transform(examples)
        #normalize data here?

    def get_test_features(self, examples):
        """
        Method to take in test text features and transform the same way as train features 
        :param examples: currently just a list of forum posts  
        """
    
        return self.allmyfeatures.transform(examples)

    def show_topX(self, num=3):
        """
        prints the top num features for the positive class and the 
        top 10 features for the negative class. 
        """
        feature_names = np.asarray([x.split("__")[1] for x in self.allmyfeatures.get_feature_names()])
        topX = np.argsort(self.estimator.coef_[0])[-num:]
        bottomX = np.argsort(self.estimator.coef_[0])[:num]
        
        print(feature_names)
        
        print("\nTop 3 features for Pos and Neg\n-------------------------")
        for fn in np.arange(1,num):
            print("Pos %i: %s %f" % (fn, feature_names[topX[-fn]], self.estimator.coef_[0,topX[-fn]]))
        for fn in np.arange(0,num-1):
            print("Neg %i: %s %f" % (fn, feature_names[bottomX[fn-1]], self.estimator.coef_[0,bottomX[fn-1]]))
            
      
    def show_misclassified(self):     

        """
        Method to show the misclassified examples i.e. False Positives and False negatives 
        """
        
        #get all the feature names
        #words = feat.allmyfeatures.get_feature_names() #####
        words = self.allmyfeatures.get_feature_names() #####
        
        # False positives
        print("\nSome misclassified examples:")
        falsepos = np.where((self.train_pred != self.y_train) & (self.train_pred == 1))[0]   #all false pos example rows
        print("\nPredicted SEP but labeled AllClear (False Pos) \n------------------------- ")

        for i in range(len(falsepos[0:10])):         #loop through falsepos examples
            weights_falsepos = []
            x = find(feat.X_train[falsepos[i]])      #find which features are used for this example
            for ii in x[1]:                          #from sparse matrix get column indices corresponding to features
                weights_falsepos.append((words[ii].split('__')[1], self.estimator.coef_[0,ii]))      #get the word and weight

            print("label: %i, prediction %i, Neg Prob: %f, Pos Prob: %f, Ex No.: %i,  example: %s " % \
                (self.y_train[falsepos[i]], self.train_pred[falsepos[i]] , self.train_pred_prob[falsepos[i]][0], \
                     self.train_pred_prob[falsepos[i]][1], falsepos[i], self.clean_examples[falsepos[i]]))
            for j in weights_falsepos:
                print(j)
                
        # False Negatives
        falseneg = np.where((self.train_pred != self.y_train) & (self.train_pred == 0))[0]
        print("\nPredicted AllClear but labeled SEP (False Neg) \n-------------------------")

        for i in range(len(falseneg[0:10])):
            weights_falseneg = []
            x = find(feat.X_train[falseneg[r]])
            for ii in x[1]:
                weights_falseneg.append((words[ii].split('__')[1], self.estimator.coef_[0,ii])) 

            print("label: %i, prediction %i, Neg Prob: %f, Pos Prob: %f, Ex No.: %i, \nexample: %s " % \
                (self.y_train[falseneg[i]], self.train_pred[falseneg[i]], self.train_pred_prob[falseneg[i]][0], \
                 self.train_pred_prob[falseneg[i]][1], falseneg[i], self.clean_examples[falseneg[i]]))
            for j in weights_falseneg:
                print(j)
        
        print("\n\n")
        print(words)
       
    def show_report(self, X, y, y_pred, y_pred_prob):
        
        """Method to show a report card of the model fit. Generalized to work for both the training and test sets
            
            X : features 
            y : labels
            y_pred : predicted labels 
            y_pred_prob : predicted probabilities

        """
        
        #print the model parameters
        print("Model Parameters \n-----------------------------------\n")
        print(self,'\n')
        
        #print the shape of the training features
        print("Features \n-----------------------------------\n")
        
        print("Shape of the Features: Num examples x Num Features")
        print(self.X_train.shape,'\n')

        #print the names of the features
        print("Feature Names: \n")
        names = self.allmyfeatures.get_feature_names()
        for n in names:
            print(n)
        print('\n')
        
        #print("\nTRAINING SET\n-------------------------------------------------------------\n")
        
        # Add confusion Matrix
        tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
        print("True Pos: %i, True Neg: %i, False Pos: %i,. False Neg: %i\n" % (tp,tn,fp,fn))

        print("\nCross Validation Metric Scores\n-----------------------------------\n")
        
        #recall
        metric1 = 'recall'
        rec_score = cross_val_score(self.estimator, X, y, \
                                    cv=self.folds,scoring=metric1)
        print(metric1)
        print(rec_score)
        print("mean: %.4f, stddev: %.4f \n" % (rec_score.mean(), rec_score.std()))
        
        #precision
        metric2 = 'precision'
        prec_score = cross_val_score(self.estimator, X, y, \
                                     cv=self.folds,scoring=metric2)
        print(metric2)
        print(prec_score)
        print("mean: %.4f, stddev: %.4f \n" % (prec_score.mean(), prec_score.std()))
    
        #matthews correlation
        metric3s = 'matthews correlation coefficient' 
        metric3 = make_scorer(matthews_corrcoef)
        mattco_score = cross_val_score(self.estimator, X, y, \
                                       cv=self.folds,scoring=metric3)
        print(metric3s)
        print(mattco_score) 
        print("mean: %.4f, stddev: %.4f \n" % (mattco_score.mean(), mattco_score.std()))
    
        #brier
        metric4s = "brier score" 
        metric4 = make_scorer(brier_score_loss)
        brier_score = cross_val_score(self.estimator, X, y, cv=self.folds,scoring=metric4)
        print(metric4s)
        print(brier_score)  
        print("mean: %.4f, stddev: %.4f \n" % (brier_score.mean(), brier_score.std()))
        
        #TSS
        def tss_score(y, y_pred):
            tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
            tss = tp / (tp + fp + fn)
            return tss
        
        metric5s = "TSS" 
        metric5 = make_scorer(tss_score)
        tss_score = cross_val_score(self.estimator, X, y, cv=self.folds,scoring=metric5)
        print(metric5s)
        print(tss_score)
        print("mean: %.4f, stddev: %.4f \n" % (tss_score.mean(), tss_score.std()))
        
        #HSS
        def hss_score(y, y_pred):
            tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
            E = ((tp + fn)*(tp + fp) + (fp + tn)*(fn + tn)) / X_train.shape[0]
            HSS = (tp + tn - E)/(X_train.shape[0] - E)
            return HSS
        
        metric6s = "HSS" 
        metric6 = make_scorer(hss_score)
        hss_score = cross_val_score(self.estimator, X, y, cv=self.folds,scoring=metric6)
        print(metric6s)
        print(hss_score)
        print("mean: %.4f, stddev: %.4f \n" % (hss_score.mean(), hss_score.std()))
        
        
        #cross validation
        #print("\nCross Validation Accuracy Scores (cross_val_predict)\n-------------------------")
        #self.y_pred = cross_val_predict(self.estimator, self.X_train, self.y_train, cv=self.folds)
        #print(self.score(self.y_pred, self.y_train))
                
        #training set score (no cross-validation)
        print('\nFull dataset (no cross-validation) Score \n-----------------------------------\n')
        
        rec1_score = recall_score(y,y_pred)
        print("Recall: %.4f" % rec1_score)
        
        prec1_score = precision_score(y,y_pred)
        print("Precision: %.4f" % prec1_score,'\n')        
        
        mattco1_score = matthews_corrcoef(y,y_pred)
        print("Mathews Corr: %.4f" % mattco1_score,'\n')        

        #calculate the mean square error
        mserr = mse(y, y_pred)
        print("MSE: %.4f" % mserr)

        #calculate the Brier score - is this the same as the MSE? And QR referenced in Balch paper?
        bsloss = bsl(y, y_pred_prob[:,1])
        print("BSL: %.4f" % bsloss)

        #Occurance rate = #SEPS / #events
        occ_rate = y.sum()/len(y)

        #Reference score of predicting all negative class
        #QR_star = 0.0324 Balch
        QR_star =  mse(y, np.zeros(len(y)))
        print("RefQuadScore: %.4f\n" % QR_star)

        #assume the QR is the same as the MSE - as stated later in Balch
        QR = mserr
        #QR = 0.0250 Balch

        #skill score 
        SS = (QR_star - QR)/QR_star
        print("SS: %.4f\n" % SS)

        FAR = fp/(tp + fp)
        POD = tp/(tp + fn)
        TSS = tp / (tp+fp+fn)
        E = ((tp + fn)*(tp + fp) + (fp + tn)*(fn + tn)) / X.shape[0]
        HSS = (tp + tn - E)/(X.shape[0] - E)

        print("FAR: %.4f" % FAR)
        print("POD: %.4f" % POD)
        print("TSS: %.4f" % TSS)
        print("HSS: %.4f" % HSS)
        
        ##ROC metrics
        #fpr, tpr, thresh = roc_curve(self.y_train, self.y_train_pred, drop_intermediate=False)

    def roc_curve(self):
        
        """plot and ROC curve"""
        
        # Initial implementation of ROC plot applied to training set
        skplt.metrics.plot_roc_curve(self.y_train, self.y_train_probas)
        
    def score(self, X, y):

        """find the accuracy score given the training data and labels"""
        #print("...In Score...")
        #print("threshold:", self.threshold)
        
        y_pred = (self.estimator.predict(X) > self.threshold).astype(int)
        
        #tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
        #return tp/(tp+fp+fn)
        
        return accuracy_score(y, y_pred)
        
    def fit(self, X, y):    #, random_state=1234):
        """
        Method to read in training data from file, and 
        train Logistic Regression classifier. 
        
        :param random_state: seed for random number generator 
        """
        
        from sklearn.linear_model import LogisticRegression 
        
        # load data 
        #self.dfTrain = pd.read_csv("AllEvtsShuffled.csv")
                      
        # get training features and labels 
        #self.X_train = self.build_train_features(self.dfTrain)    #CHANGE
        #self.y_train = np.array(self.dfTrain.sep, dtype=int)
        
        self.X_train = X 
        self.y_train = y 
        
        #print the shape of the features
        #print("Shape of the Features: Num examples x Num Features")
        #print(self.X_train.shape)
        #print("examples...:", self.X_train[0:10])

        #self.logreg.fit(self.X_train, self.y_train)
        self.estimator.fit(self.X_train, self.y_train)
        
        # make predictions on training data 
        self.y_train_pred = self.estimator.predict(self.X_train)

        #return the LogReg probabilities used to classify each example  
        self.y_train_pred_prob = self.estimator.predict_proba(self.X_train)
        

        #print report card
        print("\nTraining Set Report Card:\n----------------------------------------\n")
        print(self.show_report(self.X_train, self.y_train, self.y_train_pred, self.y_train_pred_prob))
        
    
    def predict(self, X, y):
        
        """
        Return predicted labels for exmaples X. 
 
        #### CURRENTLY THIS FUNCTION ISN'T USED - but could be called if we need predicted y vals 
        #### independently to the score function
 
        """
        
        self.X_test = X
        self.y_test = y
    
        self.y_test_pred = self.estimator.predict(self.X_test)
        self.y_test_pred_prob = self.estimator.predict_proba(self.X_test)
        
        print("\nTest Set Report Card:\n----------------------------------------\n")
        print(self.show_report(self.X_test, self.y_test, self.y_test_pred, self.y_test_pred_prob))
        
    def model_predict(self):
        """
        Method to read in test data from file, make predictions
        using trained model, and dump results to file 
        
        #### CURRENTLY THIS FUNCTION ISN'T USED - leftover from FeatEngr homework but we might need to
        #### to test on the holy grail test set
        """
        
        # read in test data 
        dfTest  = pd.read_csv("../data/spoilers/test.csv")
        
        # featurize test data 
        self.X_test = self.get_test_features(list(dfTest["sentence"]))
        
        # make predictions on test data 
        pred = self.estimator.predict(self.X_test)
        
        # dump predictions to file for submission to Kaggle  
        #pd.DataFrame({"spoiler": np.array(pred, dtype=bool)}).to_csv("prediction.csv", index=True, index_label="Id")
        
                

### Read in data

Read in the shuffled data split into training and test sets - note: neither the training or the test data is featuized at this point.


In [ ]:
#read in the shuffled dataframe of original sep+ctrl events from 1986 - 2004
orig_data = pd.read_csv("AllEvtsShuffled_1986_2004_cme.csv")

feature_data_old = orig_data[['FlrOnset','Flrmaxtime','FlrPeakFlux','FlrIntFlux2','TypeII','TypeIV','optlocation','tchianti','emchianti','FlrIntFlux','sep']]
dfTrain = feature_data_old

# to add in the SEP events from 2004 to 2017 uncomment this section
#feature_data_new = new_data[['FlrOnset','Flrmaxtime','FlrPeakFlux','FlrIntFlux2','TypeII','TypeIV','optlocation','tchianti','emchianti','FlrIntFlux','sep']]
#dfTrain = pd.concat([feature_data_old,feature_data_new])

#split dataset into training and test - not yet featurized - needs to be done after
#statement regarding how many features to use
labels = np.array(dfTrain.sep, dtype=int)
X_train0, X_test0, y_train0, y_test0 = train_test_split(dfTrain,labels,\
                            test_size=0.2, random_state=1230, stratify=labels)


****

# Logisitc Regression

*****

### Initialize SEP object and create feature data

The Logisitic Regression classifier to initialize an SEP object and generate the feature data for training and test sets.

NOTE: remember to set sepLR.allmyfeatures.set_params(LocFeatures=None,RawFeat=None) correctly here and a few cells later after the gridsearchcv. Need to find away to update the object keywords with the gridsearchcv best_params without having to reintialze the object.

In [ ]:
#Initialize SEPClass instance with an estimator of choice - here Logisitic Regression
#with estimator specific keywords 
keywords = {'random_state':1230, 'max_iter':150}

#sep_weights = {0:12,1:39} #dictionary for experimental class weights
sepLR = SEPClass(LogisticRegression(**keywords, class_weight = 'balanced', \
                C=0.9,verbose=0,solver='newton-cg'),threshold = 0.5)

#Turn off features. Options: BalchFeat, RawFeat,LocFeatures
#sepLR.allmyfeatures.set_params(LocFeatures=None,RawFeat=None)

# convert each event into features and labels 
X_train = sepLR.build_train_features(X_train0)    
y_train = y_train0

X_test = sepLR.build_train_features(X_test0)    
y_test = y_test0

#standardization
#X_train = preprocessing.scale(X_train)
#X_test = preprocessing.scale(X_test)

### Hypertune Logisitic Regression parameters with GridSearchCV

In [ ]:
# set up some scorers to work with GridSearchCV
metric3 = make_scorer(matthews_corrcoef)

def tss_score(y, y_pred):
    tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
    tss = tp / (tp + fp + fn)
    return tss

def hss_score(y, y_pred):
    tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
    E = ((tp + fn)*(tp + fp) + (fp + tn)*(fn + tn)) / X_train.shape[0]
    HSS = (tp + tn - E)/(X_train.shape[0] - E)
    return HSS

metric5 = make_scorer(tss_score)
metric6 = make_scorer(hss_score)

In [ ]:
#hyperparameter ranges
CV_dictLR = {'C':np.arange(0.0052,0.11,0.01),'class_weight':[None,'balanced'],\
           'solver':['liblinear','newton-cg','lbfgs']}

##note passing SEPClass object sep directly to gridsearchcv
cv_searchLR = GridSearchCV(estimator = sepLR.estimator, param_grid = CV_dictLR, scoring = metric5)   
cv_searchLR.fit(X_train,y_train)
cv_resultLR = cv_searchLR.best_params_
print(cv_resultLR)

### Use the tuned hyperparameters to train the LR model

Report card output with model parameters and skill score are saved into directory reports

In [ ]:
%%capture cap --no-stderr

#train the model
keywordsLR.update(cv_resultLR)
sepLR = SEPClass(LogisticRegression(**keywordsLR),threshold = 0.5)
#sepLR.allmyfeatures.set_params(LocFeatures=None,RawFeat=None)
sepLR.fit(X_train, y_train)

In [ ]:
numfeat = len(sepLR.allmyfeatures.get_feature_names())
fname = 'sepLR_nfeat%i' % (numfeat)
with open('reports/'+fname+'.txt', 'w') as f:
    f.write(cap.stdout)

### Logisitic Regression Test set

Now use the SEPClass predict method to make predictions on the test set and print out a report card

In [ ]:
#sep.predict(X_test, y_test)

### Exploration of model performance

Show the top features for both the positive and negative class

In [ ]:
sepLR.show_topX(num=3)

Miclassified events sorted by flare latitude and longitudinal position and flare SXR peak flux.

In [ ]:
# create pandas dataframe with training features and y labels plus predictions
feat_names = [n.split('__')[1] for n in sepLR.allmyfeatures.get_feature_names() ]
dfFeatTrain = pd.DataFrame(data = X_train, columns = feat_names )
dfFeatTrain['y'] = sepLR.y_train
dfFeatTrain['y_pred'] = sepLR.y_train_pred

#determine false alarms and missed events
FA = dfFeatTrain.query('y == 0 and y_pred == 1')
Miss = dfFeatTrain.query('y == 1 and y_pred == 0')


In [ ]:

#plot of flare Lat and Long vs SXR peak flux
fig, (ax1, ax2) = plt.subplots(1,2, figsize = [15,5])
fig.suptitle('SEP event Misses and False Alarms')
ax1.scatter(FA.EW, FA.NS, marker = 'o',s = FA.FlrPeakFlux/dfFeatTrain.FlrPeakFlux.max()*1000,c='b', label = 'FA')
ax1.scatter(Miss.EW, Miss.NS,s = Miss.FlrPeakFlux/dfFeatTrain.FlrPeakFlux.max()*1000, c='r', label = 'Miss')
ax1.set_title('Flare SXR Peak')
ax1.legend()

#plot of flare Lat and Long vs SXR Integrated flux
ax2.scatter(FA.EW, FA.NS, marker = 'o',s = FA.FlrIntFlux/dfFeatTrain.FlrIntFlux.max()*100,c='b'*1000, label = 'FA')
ax2.scatter(Miss.EW, Miss.NS,s = FA.FlrIntFlux/dfFeatTrain.FlrIntFlux.max()*1000, c='r', label = 'Miss')
ax2.set_title('Flare SXR Int Flux')
ax2.legend()


****
# Adaboost with Decision Trees

****

In [ ]:
#Initialize SEPClass instance with an estimator of choice - here Decision Trees

#Set up the decision tree classifier
dtc_keywords = {'max_depth':1, 'max_features':None,'random_state':1230,'min_samples_split':2,'min_samples_leaf':1}
dtc = trees(**dtc_keywords)

#Set up the SEP class with the AdaBoost Classifier
clf_keywords = {'random_state':1230}
sepDT = SEPClass(AdaBoostClassifier(dtc,**clf_keywords))

#Turn off features. Options: BalchFeat, RawFeat,LocFeatures
#sepDT.allmyfeatures.set_params(RawFeat=None)

# convert each event into features and labels 
X_train = sepDT.build_train_features(X_train0)    
y_train = y_train0

X_test = sepDT.build_train_features(X_test0)    
y_test = y_test0

#standardization
#X_train = preprocessing.scale(X_train)
#X_test = preprocessing.scale(X_test)

### Hypertune Decision Tree parameters with GridSearchCV

In [ ]:
## Tune the tree
dtc_params = {'criterion':['gini','entropy']}
dtc_search = GridSearchCV(dtc,dtc_params,scoring = metric5)
dtc_search.fit(X_train,y_train)
dtc_keywords.update(dtc_search.best_params_)
print(dtc_search.best_params_)

# Tune Adaboost
dtc = trees(**dtc_keywords)
clf = AdaBoostClassifier(dtc,**clf_keywords)
clf_params = {'n_estimators':range(50,92,2),'learning_rate':[0.01,0.1,1.0]}
clf_search = GridSearchCV(clf,clf_params, scoring = metric5)
clf_search.fit(X_train,y_train)
clf_keywords.update(clf_search.best_params_)
print(clf_search.best_params_)

### Use the tuned hyperparameters to train the Adaboost Model

In [ ]:
%%capture cap --no-stderr

#train the model
dtc = trees(**dtc_keywords)
sepDT = SEPClass(AdaBoostClassifier(dtc,**clf_keywords))
#sepLR.allmyfeatures.set_params(RawFeat=None)
sepDT.fit(X_train,y_train)

In [ ]:
numfeat = len(sepDT.allmyfeatures.get_feature_names())
fname = 'sepDT_nfeat%i' % (numfeat)
with open('reports/'+fname+'.txt', 'w') as f:
    f.write(cap.stdout)

### Adaboost Test set

Now use the SEPClass predict method to make predictions on the test set and print out a report card

In [ ]:
sepDT.predict(X_test, y_test)

### Exploration of Adaboost model performance

Show the top features for both the positive and negative class

In [ ]:
#sepDT.show_topX(num=3)

Miclassified events sorted by flare latitude and longitudinal position and flare SXR peak flux.

In [ ]:
# create pandas dataframe with training features and y labels plus predictions
feat_names = [n.split('__')[1] for n in sepDT.allmyfeatures.get_feature_names() ]
dfFeatTrain = pd.DataFrame(data = X_train, columns = feat_names )
dfFeatTrain['y'] = sepDT.y_train
dfFeatTrain['y_pred'] = sepDT.y_train_pred

#determine false alarms and missed events
FA = dfFeatTrain.query('y == 0 and y_pred == 1')
Miss = dfFeatTrain.query('y == 1 and y_pred == 0')


In [ ]:

#plot of flare Lat and Long vs SXR peak flux
fig, (ax1, ax2) = plt.subplots(1,2, figsize = [15,5])
fig.suptitle('SEP event Misses and False Alarms')
ax1.scatter(FA.EW, FA.NS, marker = 'o',s = FA.FlrPeakFlux/dfFeatTrain.FlrPeakFlux.max()*1000,c='b', label = 'FA')
ax1.scatter(Miss.EW, Miss.NS,s = Miss.FlrPeakFlux/dfFeatTrain.FlrPeakFlux.max()*1000, c='r', label = 'Miss')
ax1.set_title('Flare SXR Peak')
ax1.legend()

#plot of flare Lat and Long vs SXR Integrated flux
ax2.scatter(FA.EW, FA.NS, marker = 'o',s = FA.FlrIntFlux/dfFeatTrain.FlrIntFlux.max()*100,c='b'*1000, label = 'FA')
ax2.scatter(Miss.EW, Miss.NS,s = FA.FlrIntFlux/dfFeatTrain.FlrIntFlux.max()*1000, c='r', label = 'Miss')
ax2.set_title('Flare SXR Int Flux')
ax2.legend()


****
# Balch SEP forecast

****

SEP forecasts with machine learning techniques are compared to forecasts made using the technique outlined in the Balch 2008 paper. A statitical look up table is created for sep events and control events between 1986 and 2004. The sep event probability lookup table is 3 Dimensional with feature binning consisting of: 

5 categories of integrated flux: <0.026, 0.026-0.085, 0.085-0.275, 0.275-0.895, >0.895  
5 categories of x-ray class: <= M2, M3-M8, M9-X1,X2-X6, >= X7  
4 categories of radio type: None, type II, type IV, Both (if unknown, ignore radio event counts)  

In instances where there are not enough sample events is not enough, the dimensionality of the lookup table is collased to give the required bin statistics.

Validation metrics given in Balch 08 were mostly for the training data - where the model was validated using the same events that were used to develope the lookup table. A categorical HSS score of 0.47 was found. An effort to recover a test score was carried out by randomly splitting the data set into 2 groups, training on one half and testing on the other. 30 trials were conducted with a mean HSS of 0.46 +/- XXX. 

In [37]:
import sunpy.instr
import sunpy
import sunpy.instr
import astropy.units as u

In [38]:
def intflux_ind(INTFLX):
    """Figure out the integrated flux 'index' """
    
    if (INTFLX < 0.026): xi = 0;
    if np.logical_and((0.026  <= INTFLX), (INTFLX < 0.085)):  xi = 1;
    if np.logical_and((0.085  <= INTFLX), (INTFLX < 0.275)):  xi = 2;
    if np.logical_and((0.275  <= INTFLX), (INTFLX < 0.895)):  xi = 3;
    if (0.895  <= INTFLX): xi = 4;
        
    return xi

In [39]:
def pkflux_ind(PKFLUX):

    """figure out the x-ray class index"""

    #Flare peak flux to X-ray class
    xrcls = sunpy.instr.goes.flux_to_flareclass(PKFLUX * u.watt/u.m**2)

    #Get the class
    cxray = xrcls[0]
    
    #numerical
    nxray = int(xrcls[1])

    #Calculate index
    xci = 0;    # default
    if (cxray == 'M'):
        if (nxray <= 2): xci = 0
        if np.logical_and((nxray > 2), (nxray <= 8)): xci = 1
        if (nxray > 8): xci = 2
            
    if (cxray == 'X'):
        if (nxray <=1): xci = 2
        if np.logical_and((nxray > 1), (nxray <= 6)): xci = 3
        if (nxray > 6): xci = 4
            
    return xci

In [40]:
def radio_burst_ind(typeII, typeIV, radiopatrol):
    
    """figure out the radio burst index"""
    
    #print("# no radio ", len(np.where(radiopatrol == 0)))
    #print(radiopatrol)
    
    if (int(radiopatrol) == 0): 
        #print("no radio patrol")
        ri = -1 
    else:
        if np.logical_and((typeII == 0),(typeIV == 0)): ri =  0 # none
        if np.logical_and((typeII == 1),(typeIV == 0)): ri =  1 # type II only
        if np.logical_and((typeII == 0),(typeIV == 1)): ri =  2 # type IV only
        if np.logical_and((typeII == 1),(typeIV == 1)): ri =  3 # both

    return ri

    

In [41]:
def sep_prob(data):

    #probability of an sep event
    sep_probability = np.zeros(len(data))
    
    #loop through all events (sep + ctrl) dataframe
    for i in range(len(data)):    
    
        #determine the bin index for flare SXRintegrated flux
        xi = intflux_ind(data.FlrIntFlux2.iloc[i])
        
        #determine the bin index for the flare peak flux
        xci = pkflux_ind(data.FlrPeakFlux.iloc[i])
        
        #determine the bin for index for radio burts
        ri = radio_burst_ind(data.TypeII.iloc[i], data.TypeIV.iloc[i], data.radiopatrol.iloc[i])
        
        #sep events
        events = np.array([  
        #INTFLX < 0.026...
            [ [0, 0, 1, 0],    # X <= M2
              [1, 0, 0, 0],    # X from M3-M8	  
              [0, 0, 0, 0],    # X from M9-X1	  
              [0, 0, 0, 0],    # X from X2-X6	  
              [0, 0, 0, 0]     # X >= X7
            ], 
        #INTFLX from 0.026 to 0.085
            [ [0, 0, 0, 3],    # X <= M2
              [2, 0, 0, 3],    # X from M3-M8	  
              [1, 1, 0, 0],    # X from M9-X1	  
              [0, 0, 0, 0],    # X from X2-X6	  
              [0, 0, 0, 0]     # X >= X7
            ],
        #INTFLX from 0.085 to 0.275
            [ [0, 1, 0, 0],    # X <= M2
              [2, 2, 7, 6],    # X from M3-M8	  
              [1, 0, 0, 2],    # X from M9-X1	  
              [1, 0, 0, 3],    # X from X2-X6	  
              [0, 0, 0, 0]     # X >= X7
            ],
        #INTFLX from 0.275 to 0.895
            [ [0, 1, 0, 0],    # X <= M2
              [0, 1, 0, 3],    # X from M3-M8	  
              [2, 1, 2, 6],    # X from M9-X1	  
              [1, 0, 3,14],    # X from X2-X6	  
              [0, 1, 0, 5]     # X >= X7
            ],
        #INTFLX > 0.895
            [ [0, 0, 0, 0],    # X <= M2
              [0, 0, 0, 0],    # X from M3-M8	  
              [0, 0, 0, 0],    # X from M9-X1	  
              [1, 0, 3, 2],    # X from X2-X6	  
              [0, 0, 2, 3]     # X >= X7
            ]
            ])


        samples = np.array([ 
        # INTFLX < 0.026...
            [ [405, 26, 8, 8], # X <= M2
              [56,  16, 2, 2], # X from M3-M8	  
              [0, 0, 0, 0],    # X from M9-X1	  
              [0, 0, 0, 0],    # X from X2-X6	  
              [0, 0, 0, 0]     # X >= X7
            ],
        # INTFLX from 0.026 to 0.085
            [ [257, 15, 12, 8], # X <= M2
              [133, 24, 9, 16], # X from M3-M8	  
              [38,  11, 4, 7],     # X from M9-X1	  
              [12,   1, 0, 2],     # X from X2-X6	  
              [0,    0, 0, 0]      # X >= X7
            ],
        # INTFLX from 0.085 to 0.275
            [ [34,  2,  4,  2], # X <= M2
              [48, 11, 12, 16], # X from M3-M8	  
              [36, 10,  4, 12],    # X from M9-X1	  
              [18,  9,  4, 10],    # X from X2-X6	  
              [0,   0,  0,  0]     # X >= X7
            ],
        # INTFLX from 0.275 to 0.895
            [ [1,  0, 0,  0],    # X <= M2
              [1,  2, 2,  4],    # X from M3-M8	  
              [11, 1, 4,  8],    # X from M9-X1	  
              [14, 5, 4, 23],    # X from X2-X6	  
              [0,  1, 0,  7]     # X >= X7
            ],
        # INTFLX > 0.895
            [ [0, 0, 0,  0],    # X <= M2
              [1, 0, 0,  0],    # X from M3-M8	  
              [1, 0, 0,  0],    # X from M9-X1	  
              [2, 0, 3,  5],    # X from X2-X6	  
              [0, 0, 2, 10]     # X >= X7
            ]
        ])

        #print(events.shape)
        #print(samples.shape)
        
        
        #If radiotype is unknown or sample size is less than ten
        #it is required to collapse the radio counts
        if np.logical_or((ri == -1),(samples[xi, xci, ri] < 10)):
            #print("samples less than 10")
            #if (ri == -1):
                #print("unknown radio")
            
            samples2 = 0
            events2 = 0
            for i in range(4):
                samples2 += samples[xi, xci, i]
                events2 += events[xi, xci, i]

            # It may be necessary to collapse the xray class as well           
            if (samples2 < 10):
                #print("samples2 less than 10")
                samples3 = 0
                events3 = 0
                for i in range(5):
                    for j in range(4):
                        samples3 += samples[xi, i, j]
                        events3 += events[xi, i, j]
                        
                #return the probability estimate
                if samples3 > 0:
                    sep_probability[i] =  1.0*events3/samples3;
                else:
                    print("nan")
                    sep_probability[i] = np.nan

            sep_probability[i] = 1.0*events2/samples2;

        sep_probability[i] = 1.0*events[xi, xci, ri]/samples[xi, xci, ri]
        
    return sep_probability

In [42]:
#metrics

#TSS
def tss_score(y, y_pred):
    tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
    tss = tp / (tp + fp + fn)
    return tss


#HSS
def hss_score(y, y_pred):
    tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
    E = ((tp + fn)*(tp + fp) + (fp + tn)*(fn + tn)) / len(y)
    HSS = (tp + tn - E)/(len(y) - E)
    return HSS
        
def pod_far_pc(y, y_pred):
    tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
    
    pod = tp / (tp + fn)
    far = fp / (tp + fp)
    pc = (tp + tn)/ len(y)
    
    return pod, far, pc  


    

In [43]:
#read in the original Balch SEP event list with added CME speed and GOES T and EM
orig_data = pd.read_excel("ctrlevents.v8p.xls")

#adding y label column indicating positive and negative SEP events - SEP events have Association = ProtonFlare
orig_data['sep'] = orig_data.Association.str.contains('^Proton').astype(int)

#rename radiopatrol
orig_data.rename(columns={"Rpatrol":'radiopatrol'}, inplace=True)

#radiopatrol to int
orig_data.radiopatrol = (orig_data.radiopatrol.str.lower() == "yes").astype(int)

#convert the type II and type IV association to binary 
orig_data.TypeII = (orig_data.TypeII.str.lower() == "yes").astype(int)
orig_data.TypeIV = (orig_data.TypeIV.str.lower() == "yes").astype(int)

# Remove rows where optlocation = nan
#orig_data = orig_data[orig_data.optlocation.astype('str') != 'nan']

#shuffle the events so they are not organized 
#orig_data = shuffle(orig_data)

#save the shuffled dataframe -- commented out to prevent resaving
#orig_data.to_csv("AllEvtsShuffled_1986_2004.csv")



In [44]:
#read in the shuffled dataframe of original sep+ctrl events from 1986 - 2004
#orig_data = pd.read_csv("AllEvtsShuffled_1986_2004_cme.csv")

#pull out feature columns
data_B08 = orig_data[['FlrOnset','Flrmaxtime','FlrPeakFlux','FlrIntFlux2','TypeII','TypeIV','optlocation','FlrIntFlux','radiopatrol','sep']]
#data_B08 = orig_data[['FlrOnset','Flrmaxtime','FlrPeakFlux','FlrIntFlux2','TypeII','TypeIV','optlocation','tchianti','emchianti','FlrIntFlux','radiopatrol','sep']]
#data_B08 = data_B08.query('radiopatrol == 1')

#Use Sep object with LogReg just to featurize the data with the location
keywords = {'random_state':1230, 'max_iter':150}

sepLR = SEPClass(LogisticRegression(**keywords),threshold = 0.5)

#Turn off features. Options: BalchFeat, RawFeat,LocFeatures
sepLR.allmyfeatures.set_params(LocFeatures = None, RawFeat=None)

# convert each event into features and labels 
X_train_B08 = sepLR.build_train_features(data_B08)    
y_train_B08 = np.array(data_B08.sep, dtype=int)

feat_names = [n.split('__')[1] for n in sepLR.allmyfeatures.get_feature_names() ]
dfXtrain08 = pd.DataFrame(data = X_train_B08, columns = feat_names )
dfXtrain08['sep'] = y_train_B08
dfXtrain08['radiopatrol'] = orig_data.radiopatrol

#how manhy events occur behind the limb
#dfXtrain08.query('EW > 90.0 or EW < -90.0')

#caluclate sep event probabilities from balch 2008 table
sp = sep_prob(dfXtrain08)

#compare with actual labels
threshold = 0.3
sep_class = (sp > threshold).astype(int)

#add to data frame
dfXtrain08['sep_balch'] = sep_class.tolist()

#confusion matrix
tn, fp, fn, tp=confusion_matrix(dfXtrain08.sep, dfXtrain08.sep_balch).ravel()
print("True Pos: %i, True Neg: %i, False Pos: %i,. False Neg: %i\n" % (tp,tn,fp,fn))

tss = tss_score(dfXtrain08.sep, dfXtrain08.sep_balch)
hss = hss_score(dfXtrain08.sep, dfXtrain08.sep_balch)
pod, far, pc = pod_far_pc(dfXtrain08.sep, dfXtrain08.sep_balch)



print("TSS: %.4f, HSS:%.4f" % (tss, hss))
print("POD: %.4f, FAR:%.4f, PC:%.4f" % (pod, far, pc))

/Users/hazelbain/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:132: RuntimeWarning: invalid value encountered in double_scalars
/Users/hazelbain/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:130: RuntimeWarning: invalid value encountered in double_scalars


True Pos: 30, True Neg: 3611, False Pos: 45,. False Neg: 97

TSS: 0.1744, HSS:0.2791
POD: 0.2362, FAR:0.6000, PC:0.9625


/Users/hazelbain/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in greater


In [14]:
#tp = 60
#fn = 67
#tn = 3611
#fp = 45

#E = ((tp + fn)*(tp + fp) + (fp + tn)*(fn + tn)) / len(orig_data)
#HSS = (tp + tn - E)/(len(orig_data) - E)

#pod = tp / (tp + fn)
#far = fp / (tp + fp)
#pc = (tp + tn)/ len(orig_data)
    
#print("HSS:%.4f" % (HSS))
#print("POD: %.4f, FAR:%.4f, PC:%.4f" % (pod, far, pc))

In [ ]:
#plot
plt.scatter(np.arange(len(data)),sp)
plt.show()